In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import normalize
import os
from PIL import Image
import glob
import subprocess
import matplotlib as mpl
import gc
import matplotlib


os.makedirs("frames", exist_ok=True)
def run(phi,theta,radii):
    # ---- fibonacci sphere ----
    def fibonacci_sphere(n_points):
        """generating equal points to distributing in sphere"""
        indices = np.arange(n_points, dtype=float) + 0.5
        phi = np.arccos(1 - 2*indices/n_points)  # [0, π]
        theta = np.pi * (1 + 5**0.5) * indices  # aureo angle
        
        return phi, theta % (2*np.pi)  # etting values between 0 and 2π
    
    def cartesian_from_spherical(phi, theta, r=1.0):
        """Convert spherical coordinates to cartesians"""
        x = r * np.sin(phi) * np.cos(theta)
        y = r * np.sin(phi) * np.sin(theta)
        z = r * np.cos(phi)
        return x, y, z
    
    def spot_mask_geodesic(x, y, z, spot_center, spot_radius_rad):
        """Máscara usando distancia geodésica real con optimización"""
        # Convert the spot_center vector to a unit vector (of length 1).
        center_norm = spot_center / np.linalg.norm(spot_center)
        
        # Product
        positions = np.stack([x, y, z], axis=-1)
        norms = np.linalg.norm(positions, axis=-1, keepdims=True)
        pos_norm = positions / np.clip(norms, 1e-10, None)  #np.clip avoids division by zero (if a vector had length 0).
        
        dot_product = np.sum(pos_norm * center_norm, axis=-1) #Calculation of the Product Point
        dot_product = np.clip(dot_product, -1, 1) #Force the values to be between [-1, 1]. This is necessary because numerical errors may produce values slightly outside this range.
        
        # central_angle is the geodesic distance (in radians) between each point on the surface and the center of the spot.
        central_angle = np.arccos(dot_product)
        
        return np.exp(-(central_angle**2) / (2 * spot_radius_rad**2))
    
    #function to calculate angular velocity  
    def spot_theta(rotation_period, spot_colatitude, relative_shear):
        ''' Parameters:
        - rotation_period: rotation period of the sta
        - spot_colatitude: latitud of spot(radians)
        - relative_shear: parameter between pole and equator
        Returns:
        - angular velocity. '''
        
        latitude = np.pi / 2 - spot_colatitude
        angular_vel_equa = 2*np.pi*u.rad/rotation_period
        angular_velocity = angular_vel_equa*(1-relative_shear*np.sin(latitude)**2)
        return angular_velocity
    
    def limbdarkening(u, mu):
        ''' lineal limb darkening'''
        return (1 - u * (1 - mu))
    
    def quadratic(u1, u2, mu):
        return 1-u1*(1-mu)-u2*(1-mu)**2
        
    
    def add_spots(latitude_deg, longitude_deg, radii_deg):
        colatitude_rad = np.deg2rad(90 - latitude_deg)
        longitude_rad = np.deg2rad(longitude_deg)
        radii_rad = np.deg2rad(radii_deg)
        ang_vel = spot_theta(rotation_period, colatitude_rad, 0.4)  
        spots.append({
            'theta': longitude_rad * u.rad,
            'phi': colatitude_rad,
            'radius': radii_rad,
            'angular_velocity': ang_vel
        })
    
    def gif(input_pattern="frames/frame_%05d.png", output_gif="output.gif", 
            palette="palette.png", framerate=17):
        palette_cmd = [
            "ffmpeg", "-y", "-i", input_pattern,
            "-vf", "palettegen", palette
        ]
        gif_cmd = [
            "ffmpeg", "-y", "-framerate", str(framerate),
            "-i", input_pattern, "-i", palette,
            "-lavfi", "paletteuse", output_gif
        ]
        try:
            subprocess.run(palette_cmd, check=True)
            subprocess.run(gif_cmd, check=True)
            print(f"GIF creado: {output_gif}")
        except subprocess.CalledProcessError as e:
            print("Error en ffmpeg:", e)
    
    def flux_plot():
        '''
        Function take a list normalizing the flux, converting the list in a csv file and rename the columns
        and return a plot 
    
        '''
        frame_files = sorted(glob.glob("frames/frame_*.png"))
        fluxes = []
    
        for filename in frame_files:
            img = Image.open(filename).convert('L')  # Grayscale
            img_array = np.array(img, dtype=np.float64)
            flux_total = np.sum(img_array)
            fluxes.append(flux_total)
    
        # Normalized fluxes
        #flux_norm.append(flux_total / fluxes[i])
        flux_norm = normalize([fluxes], norm="max")[0]
        #flux_norm = np.array(fluxes)/np.max(np.array(fluxes))
        df = pd.DataFrame(flux_norm)
    
        # Creating columns
        df.index.name = 'Frame'
        df.reset_index(inplace=True)
        #changing frames for days
        df['Days'] = df['Frame'] *(cadence_time.to(u.day)).value
        df = df.rename(columns={0: 'flux_normalized'})
        df = df[['Days', 'flux_normalized']]  
    
        # saving csv
        df.to_csv(f"period_{rotation_period}_points{n_points}_obs{observing_baseline_days}_cadence{cadence_time}.csv", index=False)
    
        # plotting
        
        ax = df.plot(x="Days", y="flux_normalized", alpha=0.5, linestyle='--', color ="k")
        ax.set_xlabel("Time [days]")
        ax.set_ylabel("Normalized Flux")
        ax.set_title("Lightcurve from PNG frames")
        plt.style.use('default')
        plt.tight_layout()
        plt.savefig(f"period_{rotation_period}_days_points{n_points}_obs{observing_baseline_days}_cadence{cadence_time}.png", dpi=600)
        plt.show()
    
    # ---- Función de animación corregida ----
    def animate(i, points, base_intensity, ax_sphere, elev, azim, total_frames, vmin, vmax):
        ax_sphere.clear()
        ax_sphere.set_axis_off()
        ax_sphere.view_init(elev=elev, azim=azim)
        
        # copying the texture
    
        intensity = np.copy(base_intensity)
        
        # adding several spots
        for spot in spots:
            # Calcular nueva posición de la mancha
            theta_mov = spot['theta'] + spot['angular_velocity'] * i * cadence_time.to(u.day)
            
            # Calculating position of the spot
            spot_x = r_val * np.sin(spot['phi']) * np.cos(theta_mov.value)
            spot_y = r_val * np.sin(spot['phi']) * np.sin(theta_mov.value)
            spot_z = r_val * np.cos(spot['phi'])
            spot_center = np.array([spot_x, spot_y, spot_z])
            
            # creating a mask
            mask = spot_mask_geodesic(points[:, 0], points[:, 1], points[:, 2], 
                                     spot_center, spot['radius'])
            intensity *= (1 -  mask)  # Reducción de intensidad en manchas
        
        # scatter plot 
        sc = ax_sphere.scatter(
            points[:, 0], points[:, 1], points[:, 2], 
            #c=intensity,
            c=np.clip(intensity, 0, 1),
            cmap='gray', 
            s=1, 
            alpha=1,
            vmin=0,  # Mínimo fijo
            vmax=1.0   # Máximo fijo
        )
        
        # Configurar límites de la esfera
        max_range = r_val * 1.1
        ax_sphere.set_xlim(-max_range, max_range)
        ax_sphere.set_ylim(-max_range, max_range)
        ax_sphere.set_zlim(-max_range, max_range)
        
        print(f"Procesando frame {i+1}/{total_frames}", end='\r')
        plt.savefig(f"frames/frame_{i:05d}.png", dpi=300, bbox_inches='tight')
        
        return None
    
    # ---- main ----
    if __name__ == '__main__':
        # stellar parameter
        r_val = 1.0
        n_points = 90000
        u1 = 0.4
        u2 =0.3# limb darkening coefficients
        rotation_period = 1.0 * u.day
        
        # Point of view
        elev = 0
        azim = 0
        
        # List of spots
        spots = []
        
        #adding spots
        add_spots(phi, theta, radii)      
        #add_spots(-50, 0, 20.5)     
        #add_spots(0, 0, 20)  
        #add_spots(50, 240, 10.5) 
        
        # base lines time parameter
        observing_baseline_days = 1 * u.day
        cadence_time = 10 * u.minute
        total_frames = int((observing_baseline_days / cadence_time).decompose().value)
        
        # spherical grid with fibonacci points
        print("Generate spherical grid...")
        phi, theta = fibonacci_sphere(n_points)
        x, y, z = cartesian_from_spherical(phi, theta)
        points = np.vstack([x, y, z]).T
        
        # Calculate point of view
        elev_rad = np.deg2rad(elev) #elevation of point of view
        azim_rad = np.deg2rad(azim)#azimut of point of view
        
        v_x = np.cos(elev_rad) * np.cos(azim_rad)
        v_y = np.cos(elev_rad) * np.sin(azim_rad)
        v_z = np.sin(elev_rad)
        
        # rearrange of calculating mu parameter for limb darkening
        mu = (points[:, 0] * v_x + points[:, 1] * v_y + points[:, 2] * v_z) / r_val
        mu = np.clip(mu, 0, 1)
        #base_intensity = limbdarkening(constant, mu)# applying to the texture
        base_intensity = quadratic(u1,u2,mu) 
        #    Calculates the extreme values of the base intensity: 
        #vmin: Minimum value of intensity in the whole star.
        #vmax: Maximum value of intensity over the whole star.
        vmin=  0.0
        vmax=  1.0
    
       # Defines a reference range for color mapping that will be used consistently across all frames.
     
        
        # background configurations
        plt.style.use('dark_background')
        fig = plt.figure(figsize=(10, 8))
        ax_sphere = fig.add_subplot(111, projection='3d') 
        ax_sphere.set_axis_off()
        ax_sphere.set_box_aspect([1, 1, 1])
        
        #Límits
        max_range = r_val * 1.1
        ax_sphere.set_xlim(-max_range, max_range)
        ax_sphere.set_ylim(-max_range, max_range)
        ax_sphere.set_zlim(-max_range, max_range)
        
        # Generating animation
        print("start render...")
        for i in range(total_frames):
            animate(i, points, base_intensity, ax_sphere,  elev, azim, total_frames, vmin, vmax)
    
        
        # Create gif an light curve
        print("\nCreando GIF...")
        #gif(input_pattern="frames/frame_%05d.png", output_gif=f"period_{rotation_period}_points{n_points}_obs{observing_baseline_days}_cadence{cadence_time}_nspots{len(spots)}.gif", framerate=15)
        print("Generating ligthcurve..")
        plt.style.use('default')
        flux_plot()
    


In [ ]:
import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas as pd
import astropy.units as u
import subprocess
import glob
from PIL import Image
import os
from sklearn.preprocessing import normalize

# Función principal que ejecuta la simulación
def run(phi, theta, radii, return_flux=True):
    # Crear directorio para frames
    os.makedirs("frames", exist_ok=True)
    
    def fibonacci_sphere(n_points):
        """Generating equal points distributed on a sphere"""
        indices = np.arange(n_points, dtype=float) + 0.5
        phi_val = np.arccos(1 - 2*indices/n_points)  # [0, π]
        theta_val = np.pi * (1 + 5**0.5) * indices  # golden angle
        
        return phi_val, theta_val % (2*np.pi)  # values between 0 and 2π
    
    def cartesian_from_spherical(phi, theta, r=1.0):
        """Convert spherical coordinates to cartesian"""
        x = r * np.sin(phi) * np.cos(theta)
        y = r * np.sin(phi) * np.sin(theta)
        z = r * np.cos(phi)
        return x, y, z
    
    def spot_mask_geodesic(x, y, z, spot_center, spot_radius_rad):
        """Máscara usando distancia geodésica real"""
        center_norm = spot_center / np.linalg.norm(spot_center)
        
        positions = np.stack([x, y, z], axis=-1)
        norms = np.linalg.norm(positions, axis=-1, keepdims=True)
        pos_norm = positions / np.clip(norms, 1e-10, None)
        
        dot_product = np.sum(pos_norm * center_norm, axis=-1)
        dot_product = np.clip(dot_product, -1, 1)
        
        central_angle = np.arccos(dot_product)
        return np.exp(-(central_angle**2) / (2 * spot_radius_rad**2))
    
    def spot_theta(rotation_period, spot_colatitude, relative_shear):
        '''Calculate angular velocity'''
        latitude = np.pi / 2 - spot_colatitude
        angular_vel_equa = 2*np.pi*u.rad/rotation_period
        angular_velocity = angular_vel_equa*(1-relative_shear*np.sin(latitude)**2)
        return angular_velocity
    
    def quadratic(u1, u2, mu):
        return 1 - u1*(1-mu) - u2*(1-mu)**2
    
    def add_spots(latitude_deg, longitude_deg, radii_deg):
        colatitude_rad = np.deg2rad(90 - latitude_deg)
        longitude_rad = np.deg2rad(longitude_deg)
        radii_rad = np.deg2rad(radii_deg)
        ang_vel = spot_theta(rotation_period, colatitude_rad, 0.4)  
        spots.append({
            'theta': longitude_rad * u.rad,
            'phi': colatitude_rad,
            'radius': radii_rad,
            'angular_velocity': ang_vel
        })
    
    def animate(i, points, base_intensity, ax_sphere, elev, azim, total_frames, vmin, vmax):
        ax_sphere.clear()
        ax_sphere.set_axis_off()
        ax_sphere.view_init(elev=elev, azim=azim)
        
        intensity = np.copy(base_intensity)
        
        for spot in spots:
            theta_mov = spot['theta'] + spot['angular_velocity'] * i * cadence_time.to(u.day)
            
            spot_x = r_val * np.sin(spot['phi']) * np.cos(theta_mov.value)
            spot_y = r_val * np.sin(spot['phi']) * np.sin(theta_mov.value)
            spot_z = r_val * np.cos(spot['phi'])
            spot_center = np.array([spot_x, spot_y, spot_z])
            
            mask = spot_mask_geodesic(points[:, 0], points[:, 1], points[:, 2], 
                                     spot_center, spot['radius'])
            intensity *= (1 - mask)
        
        ax_sphere.scatter(
            points[:, 0], points[:, 1], points[:, 2], 
            c=np.clip(intensity, 0, 1),
            cmap='gray', 
            s=1, 
            alpha=1,
            vmin=0,
            vmax=1.0
        )
        
        max_range = r_val * 1.1
        ax_sphere.set_xlim(-max_range, max_range)
        ax_sphere.set_ylim(-max_range, max_range)
        ax_sphere.set_zlim(-max_range, max_range)
        
        print(f"Procesando frame {i+1}/{total_frames}", end='\r')
        plt.savefig(f"frames/frame_{i:05d}.png", dpi=150, bbox_inches='tight')
        
        return None

    # Parámetros estelares
    r_val = 1.0
    n_points = 10000  # Reducido para pruebas
    u1 = 0.4
    u2 = 0.3
    rotation_period = 1.0 * u.day
    
    # Punto de vista
    elev = 0
    azim = 0
    
    # Lista de manchas
    spots = []
    
    # Añadir manchas
    add_spots(phi, theta, radii)
    
    # Parámetros temporales
    observing_baseline_days = 1 * u.day
    cadence_time = 30 * u.minute
    total_frames = int((observing_baseline_days / cadence_time).decompose().value)
    
    # Reducir frames para pruebas
    total_frames = min(total_frames, 10)
    
    # Generar grid esférico
    phi_arr, theta_arr = fibonacci_sphere(n_points)
    x, y, z = cartesian_from_spherical(phi_arr, theta_arr)
    points = np.vstack([x, y, z]).T
    
    # Calcular punto de vista
    elev_rad = np.deg2rad(elev)
    azim_rad = np.deg2rad(azim)
    
    v_x = np.cos(elev_rad) * np.cos(azim_rad)
    v_y = np.cos(elev_rad) * np.sin(azim_rad)
    v_z = np.sin(elev_rad)
    
    # Calcular mu para limb darkening
    mu = (points[:, 0] * v_x + points[:, 1] * v_y + points[:, 2] * v_z) / r_val
    mu = np.clip(mu, 0, 1)
    base_intensity = quadratic(u1, u2, mu) 
    
    # Configuración de plot
    plt.style.use('dark_background')
    fig = plt.figure(figsize=(8, 6))
    ax_sphere = fig.add_subplot(111, projection='3d') 
    ax_sphere.set_axis_off()
    ax_sphere.set_box_aspect([1, 1, 1])
    
    max_range = r_val * 1.1
    ax_sphere.set_xlim(-max_range, max_range)
    ax_sphere.set_ylim(-max_range, max_range)
    ax_sphere.set_zlim(-max_range, max_range)
    
    # Generar animación
    print(f"Generando {total_frames} frames...")
    for i in range(total_frames):
        animate(i, points, base_intensity, ax_sphere, elev, azim, total_frames, 0, 1)
    
    plt.close(fig)
    
    # Obtener curva de luz
    frame_files = sorted(glob.glob("frames/frame_*.png"))
    fluxes = []

    for filename in frame_files:
        img = Image.open(filename).convert('L')
        img_array = np.array(img, dtype=np.float64)
        flux_total = np.sum(img_array)
        fluxes.append(flux_total)
    
    # Limpiar frames
    for filename in frame_files:
        os.remove(filename)
    
    if return_flux:
        flux_norm = np.array(fluxes) / np.max(np.array(fluxes))
        return flux_norm
    else:
        return None

# Clase MCMC para ajustar parámetros
class SpotMCMCFitter:
    def __init__(self, observed_flux):
        self.observed_flux = observed_flux
        self.time = np.arange(len(observed_flux)) * (30 * u.minute).to(u.day).value
    
    def model_function(self, params):
        phi, theta, radii = params
        model_flux = run(phi, theta, radii, return_flux=True)
        return model_flux
    
    def log_likelihood(self, params):
        try:
            model_pred = self.model_function(params)
            if len(model_pred) != len(self.observed_flux):
                return -np.inf
            
            error = 0.01
            log_like = -0.5 * np.sum(((self.observed_flux - model_pred) / error)**2)
            return log_like
        except:
            return -np.inf
    
    def log_prior(self, params):
        phi, theta, radii = params
        
        if not (-90 <= phi <= 90):
            return -np.inf
        if not (0 <= theta <= 360):
            return -np.inf
        if not (1 <= radii <= 30):
            return -np.inf
            
        return 0.0
    
    def log_probability(self, params):
        lp = self.log_prior(params)
        if not np.isfinite(lp):
            return -np.inf
        
        ll = self.log_likelihood(params)
        if not np.isfinite(ll):
            return -np.inf
            
        return lp + ll
    
    def run_mcmc(self, initial_guess, n_walkers=16, n_steps=200):
        print("Optimizando punto inicial...")
        neg_log_like = lambda p: -self.log_likelihood(p)
        result = minimize(neg_log_like, initial_guess, method='Nelder-Mead')
        best_fit = result.x
        
        print(f"Mejor ajuste inicial: phi={best_fit[0]:.2f}, theta={best_fit[1]:.2f}, radii={best_fit[2]:.2f}")
        
        ndim = 3
        pos = best_fit + 1e-2 * np.random.randn(n_walkers, ndim)
        
        print("Ejecutando MCMC...")
        sampler = emcee.EnsembleSampler(n_walkers, ndim, self.log_probability)
        sampler.run_mcmc(pos, n_steps, progress=True)
        
        return sampler, best_fit
    
    def analyze_results(self, sampler, burnin=50):
        samples = sampler.get_chain(discard=burnin, thin=5, flat=True)
        
        phi_mean, theta_mean, radii_mean = np.mean(samples, axis=0)
        phi_std, theta_std, radii_std = np.std(samples, axis=0)
        
        print(f"\nResultados MCMC:")
        print(f"Latitud (φ): {phi_mean:.2f} ± {phi_std:.2f}°")
        print(f"Longitud (θ): {theta_mean:.2f} ± {theta_std:.2f}°")
        print(f"Radio: {radii_mean:.2f} ± {radii_std:.2f}°")
        
        # Corner plot
        fig = corner.corner(samples, labels=[r"$\phi$ [°]", r"$\theta$ [°]", r"$radii$ [°]"],
                           truths=[phi_mean, theta_mean, radii_mean])
        plt.savefig("corner_plot.png", dpi=300)
        plt.close()
        
        # Chains
        fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
        labels = [r"$\phi$ [°]", r"$\theta$ [°]", r"$radii$ [°]"]
        
        for i in range(3):
            ax = axes[i]
            ax.plot(sampler.get_chain()[:, :, i], "k", alpha=0.3)
            ax.set_ylabel(labels[i])
            ax.axvline(burnin, color="red", linestyle="--")
        
        axes[-1].set_xlabel("Paso MCMC")
        plt.savefig("chains.png", dpi=300)
        plt.close()
        
        return samples

# Función principal
def main():
    # Parámetros verdaderos para simular datos
    true_params = [30.0, 45.0, 10.0]
    print("Generando datos observados...")
    observed_flux = run(true_params[0], true_params[1], true_params[2], return_flux=True)
    
    # Crear y ejecutar MCMC
    fitter = SpotMCMCFitter(observed_flux)
    sampler, best_fit = fitter.run_mcmc(initial_guess=[20.0, 30.0, 8.0], 
                                       n_walkers=16, n_steps=100)
    
    # Analizar resultados
    samples = fitter.analyze_results(sampler, burnin=20)
    
    # Mejor modelo
    best_model_params = np.mean(sampler.get_chain(discard=20, thin=5, flat=True), axis=0)
    print(f"\nMejor modelo:")
    print(f"Latitud: {best_model_params[0]:.2f}°")
    print(f"Longitud: {best_model_params[1]:.2f}°")
    print(f"Radio: {best_model_params[2]:.2f}°")
    
    # Comparar con valores verdaderos
    print(f"\nValores verdaderos:")
    print(f"Latitud: {true_params[0]:.2f}°")
    print(f"Longitud: {true_params[1]:.2f}°")
    print(f"Radio: {true_params[2]:.2f}°")

if __name__ == "__main__":
    main()

Generando datos observados...
Generando 10 frames...
Optimizando punto inicial...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames..

/Users/rubencarcamo/miniconda3/envs/tesis/lib/python3.13/site-packages/scipy/optimize/_optimize.py:851: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Procesando frame 10/10

/Users/rubencarcamo/miniconda3/envs/tesis/lib/python3.13/site-packages/scipy/optimize/_optimize.py:851: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Procesando frame 10/10

...

Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Procesando frame 10/10

/Users/rubencarcamo/miniconda3/envs/tesis/lib/python3.13/site-packages/scipy/optimize/_optimize.py:851: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Procesando frame 10/10

/Users/rubencarcamo/miniconda3/envs/tesis/lib/python3.13/site-packages/scipy/optimize/_optimize.py:851: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Procesando frame 1/10

In [ ]:
import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas as pd
import astropy.units as u
import subprocess
import glob
from PIL import Image
import os
from sklearn.preprocessing import normalize

# Función principal que ejecuta la simulación
def run(phi, theta, radii, return_flux=True):
    # Crear directorio para frames
    os.makedirs("frames", exist_ok=True)
    
    def fibonacci_sphere(n_points):
        """Generating equal points distributed on a sphere"""
        indices = np.arange(n_points, dtype=float) + 0.5
        phi_val = np.arccos(1 - 2*indices/n_points)  # [0, π]
        theta_val = np.pi * (1 + 5**0.5) * indices  # golden angle
        
        return phi_val, theta_val % (2*np.pi)  # values between 0 and 2π
    
    def cartesian_from_spherical(phi, theta, r=1.0):
        """Convert spherical coordinates to cartesian"""
        x = r * np.sin(phi) * np.cos(theta)
        y = r * np.sin(phi) * np.sin(theta)
        z = r * np.cos(phi)
        return x, y, z
    
    def spot_mask_geodesic(x, y, z, spot_center, spot_radius_rad):
        """Máscara usando distancia geodésica real"""
        center_norm = spot_center / np.linalg.norm(spot_center)
        
        positions = np.stack([x, y, z], axis=-1)
        norms = np.linalg.norm(positions, axis=-1, keepdims=True)
        pos_norm = positions / np.clip(norms, 1e-10, None)
        
        dot_product = np.sum(pos_norm * center_norm, axis=-1)
        dot_product = np.clip(dot_product, -1, 1)
        
        central_angle = np.arccos(dot_product)
        return np.exp(-(central_angle**2) / (2 * spot_radius_rad**2))
    
    def spot_theta(rotation_period, spot_colatitude, relative_shear):
        '''Calculate angular velocity'''
        latitude = np.pi / 2 - spot_colatitude
        angular_vel_equa = 2*np.pi*u.rad/rotation_period
        angular_velocity = angular_vel_equa*(1-relative_shear*np.sin(latitude)**2)
        return angular_velocity
    
    def quadratic(u1, u2, mu):
        return 1 - u1*(1-mu) - u2*(1-mu)**2
    
    def add_spots(latitude_deg, longitude_deg, radii_deg):
        colatitude_rad = np.deg2rad(90 - latitude_deg)
        longitude_rad = np.deg2rad(longitude_deg)
        radii_rad = np.deg2rad(radii_deg)
        ang_vel = spot_theta(rotation_period, colatitude_rad, 0.4)  
        spots.append({
            'theta': longitude_rad * u.rad,
            'phi': colatitude_rad,
            'radius': radii_rad,
            'angular_velocity': ang_vel
        })
    
    def animate(i, points, base_intensity, ax_sphere, elev, azim, total_frames, vmin, vmax):
        ax_sphere.clear()
        ax_sphere.set_axis_off()
        ax_sphere.view_init(elev=elev, azim=azim)
        
        intensity = np.copy(base_intensity)
        
        for spot in spots:
            theta_mov = spot['theta'] + spot['angular_velocity'] * i * cadence_time.to(u.day)
            
            spot_x = r_val * np.sin(spot['phi']) * np.cos(theta_mov.value)
            spot_y = r_val * np.sin(spot['phi']) * np.sin(theta_mov.value)
            spot_z = r_val * np.cos(spot['phi'])
            spot_center = np.array([spot_x, spot_y, spot_z])
            
            mask = spot_mask_geodesic(points[:, 0], points[:, 1], points[:, 2], 
                                     spot_center, spot['radius'])
            intensity *= (1 - mask)
        
        ax_sphere.scatter(
            points[:, 0], points[:, 1], points[:, 2], 
            c=np.clip(intensity, 0, 1),
            cmap='gray', 
            s=1, 
            alpha=1,
            vmin=0,
            vmax=1.0
        )
        
        max_range = r_val * 1.1
        ax_sphere.set_xlim(-max_range, max_range)
        ax_sphere.set_ylim(-max_range, max_range)
        ax_sphere.set_zlim(-max_range, max_range)
        
        print(f"Procesando frame {i+1}/{total_frames}", end='\r')
        plt.savefig(f"frames/frame_{i:05d}.png", dpi=150, bbox_inches='tight')
        
        return None

    # Parámetros estelares
    r_val = 1.0
    n_points = 10000  # Reducido para pruebas
    u1 = 0.4
    u2 = 0.3
    rotation_period = 1.0 * u.day
    
    # Punto de vista
    elev = 0
    azim = 0
    
    # Lista de manchas
    spots = []
    
    # Añadir manchas
    add_spots(phi, theta, radii)
    
    # Parámetros temporales
    observing_baseline_days = 1 * u.day
    cadence_time = 30 * u.minute
    total_frames = int((observing_baseline_days / cadence_time).decompose().value)
    
    # Reducir frames para pruebas
    total_frames = min(total_frames, 10)
    
    # Generar grid esférico
    phi_arr, theta_arr = fibonacci_sphere(n_points)
    x, y, z = cartesian_from_spherical(phi_arr, theta_arr)
    points = np.vstack([x, y, z]).T
    
    # Calcular punto de vista
    elev_rad = np.deg2rad(elev)
    azim_rad = np.deg2rad(azim)
    
    v_x = np.cos(elev_rad) * np.cos(azim_rad)
    v_y = np.cos(elev_rad) * np.sin(azim_rad)
    v_z = np.sin(elev_rad)
    
    # Calcular mu para limb darkening
    mu = (points[:, 0] * v_x + points[:, 1] * v_y + points[:, 2] * v_z) / r_val
    mu = np.clip(mu, 0, 1)
    base_intensity = quadratic(u1, u2, mu) 
    
    # Configuración de plot
    plt.style.use('dark_background')
    fig = plt.figure(figsize=(8, 6))
    ax_sphere = fig.add_subplot(111, projection='3d') 
    ax_sphere.set_axis_off()
    ax_sphere.set_box_aspect([1, 1, 1])
    
    max_range = r_val * 1.1
    ax_sphere.set_xlim(-max_range, max_range)
    ax_sphere.set_ylim(-max_range, max_range)
    ax_sphere.set_zlim(-max_range, max_range)
    
    # Generar animación
    print(f"Generando {total_frames} frames...")
    for i in range(total_frames):
        animate(i, points, base_intensity, ax_sphere, elev, azim, total_frames, 0, 1)
    
    plt.close(fig)
    
    # Obtener curva de luz
    frame_files = sorted(glob.glob("frames/frame_*.png"))
    fluxes = []

    for filename in frame_files:
        img = Image.open(filename).convert('L')
        img_array = np.array(img, dtype=np.float64)
        flux_total = np.sum(img_array)
        fluxes.append(flux_total)
    
    # Limpiar frames
    for filename in frame_files:
        try:
            os.remove(filename)
        except:
            pass
    
    if return_flux:
        flux_norm = np.array(fluxes) / np.max(np.array(fluxes))
        return flux_norm
    else:
        return None

# Clase MCMC para ajustar parámetros
class SpotMCMCFitter:
    def __init__(self, observed_flux):
        self.observed_flux = observed_flux
        self.time = np.arange(len(observed_flux)) * (30 * u.minute).to(u.day).value
    
    def model_function(self, params):
        try:
            phi, theta, radii = params
            
            # Verificar parámetros antes de ejecutar
            if not (-90 <= phi <= 90) or not (0 <= theta <= 360) or not (1 <= radii <= 30):
                return np.ones_like(self.observed_flux) * np.nan
            
            model_flux = run(phi, theta, radii, return_flux=True)
            
            # Verificar resultado
            if model_flux is None or len(model_flux) != len(self.observed_flux):
                return np.ones_like(self.observed_flux) * np.nan
                
            return model_flux
        except Exception as e:
            print(f"Error en model_function: {e}")
            return np.ones_like(self.observed_flux) * np.nan
    
    def log_likelihood(self, params):
        try:
            model_pred = self.model_function(params)
            
            # Verificar si hay problemas en la predicción
            if (np.any(np.isnan(model_pred)) or 
                np.any(np.isinf(model_pred)) or 
                len(model_pred) != len(self.observed_flux)):
                return -1e10  # Valor grande negativo en lugar de -inf
            
            error = 0.01
            log_like = -0.5 * np.sum(((self.observed_flux - model_pred) / error)**2)
            
            # Asegurar que no sea -infinito
            if np.isinf(log_like) or np.isnan(log_like):
                return -1e10
                
            return log_like
        except Exception as e:
            print(f"Error en log_likelihood: {e}")
            return -1e10
    
    def log_prior(self, params):
        phi, theta, radii = params
        
        # Rangos más conservadores con márgenes de seguridad
        if not (-85 <= phi <= 85):  # Un poco dentro de los límites
            return -1e10
        if not (5 <= theta <= 355):  # Evitar límites exactos
            return -1e10
        if not (2 <= radii <= 25):   # Rango más seguro
            return -1e10
            
        return 0.0
    
    def log_probability(self, params):
        lp = self.log_prior(params)
        if not np.isfinite(lp):
            return -1e10
        
        ll = self.log_likelihood(params)
        if not np.isfinite(ll):
            return -1e10
            
        return lp + ll
    
    def run_mcmc(self, initial_guess, n_walkers=16, n_steps=100):
        print("Optimizando punto inicial...")
        
        # Función objetivo con manejo de errores
        def safe_neg_log_like(p):
            try:
                result = -self.log_likelihood(p)
                if np.isinf(result) or np.isnan(result):
                    return 1e10  # Valor grande positivo para minimización
                return result
            except:
                return 1e10
        
        # Usar método más robusto
        result = minimize(safe_neg_log_like, initial_guess, 
                         method='Powell',  # Más robusto que Nelder-Mead
                         options={'disp': True, 'maxiter': 30})
        
        if result.success:
            best_fit = result.x
            print(f"Mejor ajuste inicial: phi={best_fit[0]:.2f}, theta={best_fit[1]:.2f}, radii={best_fit[2]:.2f}")
        else:
            print("Optimización inicial falló, usando guess inicial")
            best_fit = initial_guess
        
        ndim = 3
        pos = best_fit + 1e-2 * np.random.randn(n_walkers, ndim)
        
        print("Ejecutando MCMC...")
        sampler = emcee.EnsembleSampler(n_walkers, ndim, self.log_probability)
        sampler.run_mcmc(pos, n_steps, progress=True)
        
        return sampler, best_fit
    
    def analyze_results(self, sampler, burnin=20):
        samples = sampler.get_chain(discard=burnin, thin=5, flat=True)
        
        phi_mean, theta_mean, radii_mean = np.mean(samples, axis=0)
        phi_std, theta_std, radii_std = np.std(samples, axis=0)
        
        print(f"\nResultados MCMC:")
        print(f"Latitud (φ): {phi_mean:.2f} ± {phi_std:.2f}°")
        print(f"Longitud (θ): {theta_mean:.2f} ± {theta_std:.2f}°")
        print(f"Radio: {radii_mean:.2f} ± {radii_std:.2f}°")
        
        # Corner plot
        fig = corner.corner(samples, labels=[r"$\phi$ [°]", r"$\theta$ [°]", r"$radii$ [°]"],
                           truths=[phi_mean, theta_mean, radii_mean])
        plt.savefig("corner_plot.png", dpi=300)
        plt.close()
        
        # Chains
        fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
        labels = [r"$\phi$ [°]", r"$\theta$ [°]", r"$radii$ [°]"]
        
        for i in range(3):
            ax = axes[i]
            ax.plot(sampler.get_chain()[:, :, i], "k", alpha=0.3)
            ax.set_ylabel(labels[i])
            ax.axvline(burnin, color="red", linestyle="--")
        
        axes[-1].set_xlabel("Paso MCMC")
        plt.savefig("chains.png", dpi=300)
        plt.close()
        
        return samples

# Función principal
def main():
    # Parámetros verdaderos para simular datos
    true_params = [30.0, 45.0, 10.0]
    print("Generando datos observados...")
    observed_flux = run(true_params[0], true_params[1], true_params[2], return_flux=True)
    
    # Crear y ejecutar MCMC
    fitter = SpotMCMCFitter(observed_flux)
    sampler, best_fit = fitter.run_mcmc(initial_guess=[20.0, 30.0, 8.0], 
                                       n_walkers=16, n_steps=100)
    
    # Analizar resultados
    samples = fitter.analyze_results(sampler, burnin=20)
    
    # Mejor modelo
    best_model_params = np.mean(sampler.get_chain(discard=20, thin=5, flat=True), axis=0)
    print(f"\nMejor modelo:")
    print(f"Latitud: {best_model_params[0]:.2f}°")
    print(f"Longitud: {best_model_params[1]:.2f}°")
    print(f"Radio: {best_model_params[2]:.2f}°")
    
    # Comparar con valores verdaderos
    print(f"\nValores verdaderos:")
    print(f"Latitud: {true_params[0]:.2f}°")
    print(f"Longitud: {true_params[1]:.2f}°")
    print(f"Radio: {true_params[2]:.2f}°")
    
    # Graficar comparación de curvas de luz
    plt.figure(figsize=(10, 6))
    time = np.arange(len(observed_flux)) * 0.5  # 0.5 días por frame
    
    # Curva observada
    plt.plot(time, observed_flux, 'ko-', label='Observado', alpha=0.7)
    
    # Curva del mejor modelo
    best_model_flux = run(best_model_params[0], best_model_params[1], best_model_params[2], return_flux=True)
    plt.plot(time, best_model_flux, 'r-', label='Mejor modelo MCMC', linewidth=2)
    
    plt.xlabel('Tiempo (días)')
    plt.ylabel('Flujo normalizado')
    plt.title('Comparación de curvas de luz')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('comparacion_curvas_luz.png', dpi=300, bbox_inches='tight')
    plt.show()

if __name__ == "__main__":
    main()

Generando datos observados...
Generando 10 frames...
Optimizando punto inicial...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Generando 10 frames...
Procesando frame 9/10